# **Ứng dụng của học máy trong cảnh báo nguy cơ mắc bệnh tim mạch vành**

<img src='242452624.jpg'>

## **1. Những mã nguồn sử dụng trong quá trình phát triển mô hình**

In [1]:
# mã nguồn hỗ trợ nhập dữ liệu từ bộ nhớ
import pandas as pd
import numpy as np
# vẽ biểu đồ
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# utils
from collections import Counter
import sklearn
# xử lí dữ liệu
from sklearn.preprocessing import StandardScaler
# phân tách dữ liệu
from sklearn.model_selection import train_test_split
# phát triển mô hình
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


## **2. Nhập và xử lí dữ liệu**

### **2.1. Nhập dữ liệu từ bộ nhớ**

In [2]:
data = pd.read_csv('./framingham.csv')
data.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
data.isnull().sum() # Đếm số lượng dữ liệu thiếu theo từng cột

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

**Note:** Có thể thấy dữ liệu được công bố có nhiều dữ liệu trống (Null data), không thể sử lí đơn thuần bằng cách điền dữ liệu trung lập (0) vào phần còn thiếu.

In [4]:
data.describe()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,4240.000000,4240.000000,4135.000000,4240.000000,4211.000000,4187.000000,4240.000000,4240.000000,4240.000000,4190.000000,4240.000000,4240.000000,4221.000000,4239.000000,3852.000000,4240.000000
mean,0.429245,49.580189,1.979444,0.494104,9.005937,0.029615,0.005896,0.310613,0.025708,236.699523,132.354599,82.897759,25.800801,75.878981,81.963655,0.151887
std,0.495027,8.572942,1.019791,0.500024,11.922462,0.169544,0.076569,0.462799,0.158280,44.591284,22.033300,11.910394,4.079840,12.025348,23.954335,0.358953
min,0.000000,32.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,107.000000,83.500000,48.000000,15.540000,44.000000,40.000000,0.000000
25%,0.000000,42.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,206.000000,117.000000,75.000000,23.070000,68.000000,71.000000,0.000000
50%,0.000000,49.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,234.000000,128.000000,82.000000,25.400000,75.000000,78.000000,0.000000
75%,1.000000,56.000000,3.000000,1.000000,20.000000,0.000000,0.000000,1.000000,0.000000,263.000000,144.000000,90.000000,28.040000,83.000000,87.000000,0.000000
max,1.000000,70.000000,4.000000,1.000000,70.000000,1.000000,1.000000,1.000000,1.000000,696.000000,295.000000,142.500000,56.800000,143.000000,394.000000,1.000000



### **2.2. Xử lí dữ liệu thiếu trong bộ dữ liệu gốc**

Nhóm quyết định dùng [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html?highlight=simpleimputer#sklearn.impute.SimpleImputer) từ mã nguồn scikit-learn để xử lí những dữ liệu thiếu xót trong bộ dữ liệu gốc do NHLBI (Viện Tim, Phổi và Máu Quốc gia thuộc cơ quan chủ quản Bộ Y tế và Dịch vụ Nhân sinh Hoa Kỳ) công bố.

Phương pháp mà nhóm chọn khi phát triển các mô hình là dùng trung bình cộng của những dữ liệu còn lại cùng cột.

In [5]:
from sklearn.impute import SimpleImputer
data_fix = data.copy()

mean_imputer = SimpleImputer(strategy='mean') 
data_fix.iloc[:,:] = mean_imputer.fit_transform(data_fix)
data_fix.isnull().sum()        # Kiểm tra số lượng dữ liệu chưa được điền


male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

In [6]:
print(data_fix)

      male   age  education  currentSmoker  cigsPerDay    BPMeds  \
0      1.0  39.0        4.0            0.0         0.0  0.000000   
1      0.0  46.0        2.0            0.0         0.0  0.000000   
2      1.0  48.0        1.0            1.0        20.0  0.000000   
3      0.0  61.0        3.0            1.0        30.0  0.000000   
4      0.0  46.0        3.0            1.0        23.0  0.000000   
...    ...   ...        ...            ...         ...       ...   
4235   0.0  48.0        2.0            1.0        20.0  0.029615   
4236   0.0  44.0        1.0            1.0        15.0  0.000000   
4237   0.0  52.0        2.0            0.0         0.0  0.000000   
4238   1.0  40.0        3.0            0.0         0.0  0.000000   
4239   0.0  39.0        3.0            1.0        30.0  0.000000   

      prevalentStroke  prevalentHyp  diabetes  totChol  sysBP  diaBP    BMI  \
0                 0.0           0.0       0.0    195.0  106.0   70.0  26.97   
1                 0.0    

## **3. Chia dữ liệu thành tập huấn luyện và tập đánh giá**

In [7]:
y = data_fix["TenYearCHD"]
X = data_fix.drop("TenYearCHD",axis=1)

**Note:** Dữ liệu nên được chuẩn hóa trước khi chia thành tập huấn luyện và tập đánh giá. Mã nguồn scikit-learn có công cụ [preprocessing.normalize](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html?highlight=normalize#sklearn.preprocessing.normalize) để hỗ trợ việc này. Điều này được tiến sĩ Joaquin Sevilla và cộng sự khẳng định trong [nghiên cứu](https://www.researchgate.net/publication/3135573_Importance_of_input_data_normalization_for_the_application_of_neural_networks_to_complex_industrial_problems) của mình vào năm 1997.


In [8]:
X = sklearn.preprocessing.normalize(X, norm='l2', axis=1, copy=True, return_norm=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)

In [9]:
print(X_train)

[[0.         0.1743065  0.01340819 ... 0.07548812 0.2514036  0.27474611]
 [0.00311213 0.11514863 0.00622425 ... 0.08589465 0.19606388 0.17739113]
 [0.00361533 0.20607362 0.00361533 ... 0.13788856 0.24584221 0.3398407 ]
 ...
 [0.00447479 0.17451668 0.01342436 ... 0.10949803 0.31323507 0.36245772]
 [0.         0.13231932 0.00232139 ... 0.05634018 0.17410437 0.18803272]
 [0.         0.12951942 0.00647597 ... 0.0860009  0.24284892 0.23637295]]


In [10]:
print(y_test.unique())
counter = Counter(y_train)
print(counter[0], counter[1])

[0. 1.]
2520 448


In [11]:
class_weight = {0: 1.,
                1: counter[0]/counter[1]}

Có thể thấy trong tập dữ liệu huấn luyện dữ liệu âm tính nhiều hơn khoảng gần 6 lần so với dữ liệu dương tính. Nếu không xử lí ở điểm này, mô hình sẽ có xu hướng cố gắng cho ra rất nhiều kết quả âm tính. Để mô phỏng quá trình học của con người, nhóm quyết định "ép buộc" mô hình phải tập trung nhiều hơn vào những mẫu dương tính.

`class_weight` dùng cho quá trình này.

## **4. Phát triển mô hình dự đoán**

**Note:** Ngoài độ chính xác tính theo phần trăm (accuracy trong bảo báo cáo) thì đối với bộ dữ liệu không cân bằng giữa số lượng mẫu dương tính và âm tính thì rất cần thiết phải có phương pháp tính điểm F1. Phương pháp này được tiến sĩ Marina Sokolova (University of Ottawa) và cộng sự giới thiệu lần đầu vào năm 2006 trong [bài nghiên cứu](https://www.researchgate.net/publication/225215404_Beyond_Accuracy_F-Score_and_ROC_A_Family_of_Discriminant_Measures_for_Performance_Evaluation) của mình.

# **4.1. Linear Regression**

In [12]:
m1 = 'Logistic Regression'
lr = LogisticRegression(class_weight = class_weight)
model = lr.fit(X_train, y_train)
lr_predict = lr.predict(X_test)
lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_f1 = sklearn.metrics.f1_score(y_test, lr_predict, average='macro')
lr_acc_score = accuracy_score(y_test, lr_predict)
print("confusion matrix:")
print(lr_conf_matrix)
print("\n")
print("Accuracy of Logistic Regression: {:.5}".format(lr_acc_score*100),'\n')
print("F1 Score of Logistic Regression: {:.5}".format(lr_f1*100),'\n')
print(classification_report(y_test,lr_predict))


confusion matrix:
[[711 365]
 [ 77 119]]


Accuracy of Logistic Regression: 65.252 

F1 Score of Logistic Regression: 55.644 

              precision    recall  f1-score   support

         0.0       0.90      0.66      0.76      1076
         1.0       0.25      0.61      0.35       196

    accuracy                           0.65      1272
   macro avg       0.57      0.63      0.56      1272
weighted avg       0.80      0.65      0.70      1272



## ***4.2. Random Forest Classfier*** 

In [13]:
m2 = 'Random Forest Classfier'
rf = RandomForestClassifier(random_state=None,max_depth=8, class_weight=class_weight)
rf.fit(X_train,y_train)
rf_predicted = rf.predict(X_test)
rf_f1 = sklearn.metrics.f1_score(y_test, rf_predicted, average='macro')

rf_conf_matrix = confusion_matrix(y_test, rf_predicted)
rf_acc_score = accuracy_score(y_test, rf_predicted)
print("confusion matrix:")
print(rf_conf_matrix)
print("\n")
print("Accuracy of Random Forest Classifier: {:.5}".format(rf_acc_score*100),'\n')
print("F1 Score of Random Forest Classifier: {:.5}".format(rf_f1*100),'\n')
print(classification_report(y_test,rf_predicted))

confusion matrix:
[[966 110]
 [143  53]]


Accuracy of Random Forest Classifier: 80.11 

F1 Score of Random Forest Classifier: 58.974 

              precision    recall  f1-score   support

         0.0       0.87      0.90      0.88      1076
         1.0       0.33      0.27      0.30       196

    accuracy                           0.80      1272
   macro avg       0.60      0.58      0.59      1272
weighted avg       0.79      0.80      0.79      1272



## ***4.3. Random Forest Classifier without class weight***

In [14]:
m3 = 'Random Forest Classfier'
rf_ = RandomForestClassifier(random_state=None,max_depth=8)
rf_.fit(X_train,y_train)
rf_predicted_ = rf_.predict(X_test)
rf_f1_ = sklearn.metrics.f1_score(y_test, rf_predicted_, average='macro')

rf_conf_matrix_ = confusion_matrix(y_test, rf_predicted)
rf_acc_score_ = accuracy_score(y_test, rf_predicted)
print("confussion matrix:")
print(rf_conf_matrix_)
print("\n")
print("Accuracy of Random Forest Classifier: {:.5}".format(rf_acc_score_*100),'\n')
print("F1 Score of Random Forest Classifier: {:.5}".format(rf_f1_*100),'\n')
print(classification_report(y_test,rf_predicted_))

confussion matrix:
[[966 110]
 [143  53]]


Accuracy of Random Forest Classifier: 80.11 

F1 Score of Random Forest Classifier: 47.707 

              precision    recall  f1-score   support

         0.0       0.85      0.99      0.92      1076
         1.0       0.40      0.02      0.04       196

    accuracy                           0.84      1272
   macro avg       0.62      0.51      0.48      1272
weighted avg       0.78      0.84      0.78      1272



Có thể thấy nếu không có `class_weight` mô hình Random Forest (mô hình đã làm rất tốt trước đó) gặp rất nhiều khó khăn để dự đoán được những mẫu dương tính trong tập đánh giá, dẫn đến ảnh hưởng rất lớn đến điểm F1.

In [22]:
import pickle

#Serialize mô hình
with open('trained_model.pkl', 'wb') as f:
    pickle.dump(rf, f)

#De-serialize mô hình
with open('trained_model.pkl', 'rb') as f:
    clf_loaded = pickle.load(f)
    
    

In [23]:
#Check the pickle file by inputing the variables
model = pickle.load(open('trained_model.pkl','rb'))
print(model.predict_proba([[0, 61, 3.0, 1, 30.0, 0.0, 0, 1, 0, 225.0, 150.0, 95.0, 28.58, 65.0, 103.0]]))

[[0.75653796 0.24346204]]
